In [ ]:
import matplotlib.pyplot as plt
import plot_figures
from jax import vmap, jit, random, numpy as jnp

import sys
import os

# Add the scripts folder to sys.path
sys.path.append(os.path.abspath("../"))

# Now you can import my_script
import spatial_embedings
import extra_initializers

In [ ]:
grid = spatial_embedings.twod_grid(10, 10)

In [ ]:
def pairwise_sq_distance(x, y):
    return spatial_embedings.sq_distance(x[0], x[1], y[0], y[1])

# Apply the pairwise squared distance function across all points
sq_pairwise_distances = vmap(lambda p: vmap(lambda q: pairwise_sq_distance(p, q))(grid))(grid)

In [ ]:
sigma = 0.012
key = random.PRNGKey(14515414554512)
grid_shape = (10,10)
n_rec=100
connect_grid = jnp.zeros(100)
fixed_cell_ind = [4,4]
iter =10000
subkey_loc, key = random.split(key, 2)
cells_loc = extra_initializers.initialize_neurons_position(gridshape=grid_shape, key=subkey_loc, n_rec=n_rec)()
index = jnp.where(jnp.all(cells_loc == jnp.array((4, 4)), axis=1))[0]
index
cells_loc[43]
index = jnp.where(jnp.all(cells_loc == jnp.array((4, 4)), axis=1))[0]
rec_cell_loc_ind = jnp.lexsort((cells_loc[:, 1], cells_loc[:, 0])) # sort by row, then column 
rec_cell_loc_ind[44]
index
jnp.array([44])

In [ ]:


def run_sim(connect_grid, key, cells_loc): 
    subkey, key = random.split(key, 2)    
    M =extra_initializers.initialize_connectivity_mask(connectivity_rec_layer="local", gridshape=grid_shape,
                                                neuron_indices=cells_loc, key=subkey,
                                                n_rec=n_rec, sigma=sigma, sparsity=0.1
    )()

    sorted_recurrent_weights =  M[jnp.ix_(rec_cell_loc_ind, rec_cell_loc_ind)]# (n_rec, n_rec) sorts both rows and columns 
    connection = jnp.where(sorted_recurrent_weights[jnp.array([44]), :] == 1.)[1]
    new_connect_grid = connect_grid.at[connection].add(1/iter)
    return new_connect_grid


In [ ]:
for i in range(iter):
    subkey, key = random.split(key, 2)
    connect_grid = run_sim(connect_grid, subkey,cells_loc )
    if i % 100 == 0:
        print(i)

In [ ]:
connect_grid

In [ ]:
fig,ax = plt.subplots(figsize=(6, 6))
prob_grid = connect_grid.reshape(10, 10)
prob_grid = prob_grid.at[4, 4].set(jnp.nan)
cax = ax.imshow(prob_grid, cmap="viridis", origin="upper", vmin=0, vmax=0.45)
ax.invert_yaxis()  # Optional, to match usual grid orientation